In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import IO
import seaborn as sns

df = pd.read_csv('./Data/kddcup99/kddcup.data_10_percent/kddcup.data_10_percent')

X = df.iloc[:,:41]
Y = df.iloc[:,41]
X.columns=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate"]

prt_map ={}
srv_map = {}
flag_map = {}
value = 0
for i in X['protocol_type'].unique():
    prt_map.update({i:value})
    value+=1

value = 0
for i in X['service'].unique():
    srv_map.update({i:value})
    value+=1
    
value = 0
for i in X['flag'].unique():
    flag_map.update({i:value})
    value+=1

X['protocol_type'] = X['protocol_type'].map(prt_map)
X['service'] = X['service'].map(srv_map)
X['flag'] = X['flag'].map(flag_map)

Y = Y.str.replace('.','')
attacks_types = {
'normal': 'normal',
'back': 'dos',
'buffer_overflow': 'u2r',
'ftp_write': 'r2l',
'guess_passwd': 'r2l',
'imap': 'r2l',
'ipsweep': 'probe',
'land': 'dos',
'loadmodule': 'u2r',
'multihop': 'r2l',
'neptune': 'dos',
'nmap': 'probe',
'perl': 'u2r',
'phf': 'r2l',
'pod': 'dos',
'portsweep': 'probe',
'rootkit': 'u2r',
'satan': 'probe',
'smurf': 'dos',
'spy': 'r2l',
'teardrop': 'dos',
'warezclient': 'r2l',
'warezmaster': 'r2l',
}
Y = Y.map(attacks_types)

map_y_label={}
value=0
for i in Y.unique():
    map_y_label.update({i:value})
    value+=1
Y = Y.map(map_y_label)

X = X[[col for col in X if X[col].nunique() > 1]]# keep columns where there are more than 1 unique values


In [9]:
corr = X.corr()
data = (corr.rename_axis('row_id')
         .reset_index()
         .melt(id_vars='row_id', var_name='column_id')
         .to_dict('records'))
data

[{'row_id': 'duration', 'column_id': 'duration', 'value': 1.0},
 {'row_id': 'protocol_type',
  'column_id': 'duration',
  'value': -0.023715344778526094},
 {'row_id': 'service', 'column_id': 'duration', 'value': 0.02718440415977243},
 {'row_id': 'flag', 'column_id': 'duration', 'value': -0.006068169050883122},
 {'row_id': 'src_bytes',
  'column_id': 'duration',
  'value': 0.004258229894344121},
 {'row_id': 'dst_bytes',
  'column_id': 'duration',
  'value': 0.005439553637566612},
 {'row_id': 'land', 'column_id': 'duration', 'value': -0.00045240338208928283},
 {'row_id': 'wrong_fragment',
  'column_id': 'duration',
  'value': -0.0032350400429680295},
 {'row_id': 'urgent', 'column_id': 'duration', 'value': 0.0037859962046768243},
 {'row_id': 'hot', 'column_id': 'duration', 'value': 0.013212921784595748},
 {'row_id': 'num_failed_logins',
  'column_id': 'duration',
  'value': 0.005238858597699668},
 {'row_id': 'logged_in',
  'column_id': 'duration',
  'value': -0.01726492982801701},
 {'row_

In [55]:
list_corr = []
for d in data:
    if d['value']>0.97 and d['row_id']!=d['column_id']:
        list_corr.append([d['column_id'],d['row_id']])

In [56]:
len(list_corr)

28

In [57]:
for i in list_corr:
    for j in list_corr:
        if(i[0]==j[1] and i[1]==j[0]):
            list_corr.remove(i)
list_corr

[['num_root', 'num_compromised'],
 ['serror_rate', 'dst_host_serror_rate'],
 ['srv_serror_rate', 'serror_rate'],
 ['srv_serror_rate', 'dst_host_srv_serror_rate'],
 ['rerror_rate', 'dst_host_rerror_rate'],
 ['srv_rerror_rate', 'rerror_rate'],
 ['srv_rerror_rate', 'dst_host_srv_rerror_rate'],
 ['dst_host_same_srv_rate', 'dst_host_srv_count'],
 ['dst_host_serror_rate', 'srv_serror_rate'],
 ['dst_host_srv_serror_rate', 'serror_rate'],
 ['dst_host_srv_serror_rate', 'dst_host_serror_rate'],
 ['dst_host_rerror_rate', 'srv_rerror_rate'],
 ['dst_host_srv_rerror_rate', 'rerror_rate'],
 ['dst_host_srv_rerror_rate', 'dst_host_rerror_rate']]

In [62]:
fl = []
for i in list_corr:
    for j in i:
        fl.append(j)
fl
len(fl)

28

In [63]:
pd.value_counts(fl)

serror_rate                 3
dst_host_serror_rate        3
srv_serror_rate             3
dst_host_srv_serror_rate    3
rerror_rate                 3
dst_host_rerror_rate        3
srv_rerror_rate             3
dst_host_srv_rerror_rate    3
num_root                    1
num_compromised             1
dst_host_same_srv_rate      1
dst_host_srv_count          1
Name: count, dtype: int64

In [65]:
len(pd.unique(fl))

12